# This is the pcdet-free version of beamng 3d detection

# Pcdet解耦版本的beamng 3d目标检测

## Install-安装

- create environment and activate it in anaconda-创建并激活anaconda环境

  ```bash
  conda env create -f ./environment.yaml
  conda activate env_beampy_torch
  ```

- install spconv from https://github.com/traveller59/spconv 安装spconv

  for example 例如:

  depends on your version of cuda 注意取决于你的cuda版本

  ```
  pip install spconv-cu118
  ```

## Launch-启动

Run all the code cell in order while following the instructions 按照步骤每个代码块逐个启动即可

In [1]:
from beamng_control import beamng_server, beamng_client
from utils import *
from utils.mods.post_processing import mono_label_distance_tracker
import open3d as o3d
import numpy as np
import torch

import time
from IPython.display import clear_output, display

2023-09-19 11:07:55,059 |INFO     |beamngpy                      |Started BeamNGpy logging.
2023-09-19 11:07:59,188 |WARNING  |py.warnings                   |<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility

2023-09-19 11:07:59,188 |WARNING  |py.warnings                   |<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logger = create_logger()
LAUNCH_SERVER = True
REMOTE_IP = "192.168.31.13" # only for server == False

In [3]:
if LAUNCH_SERVER:
    server_a = beamng_server() 
client_b = beamng_client(host="127.0.0.1" if LAUNCH_SERVER else REMOTE_IP, logger=logger)

In [4]:
if LAUNCH_SERVER:
    server_a.init_server()
    server_a.launch_server()

2023-09-19 11:08:02,519 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_beampy_torch\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=2432, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 6639)>

2023-09-19 11:08:02,519 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_beampy_torch\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=2432, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 6639)>

2023-09-19 11:08:02,522 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2023-09-19 11:08:02,622 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2023-09-19 11:08:12,623 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (127.0.0.1, 64256)
2023-09-19 11:08:12,729 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
202

In [5]:
client_b.init_client()
client_b.launch_client()

2023-09-19 11:08:13,743 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-09-19 11:08:13,744 |INFO     |beamngpy.BeamNGpy             |BeamNGpy successfully connected to existing BeamNG instance.


In [6]:
if LAUNCH_SERVER:
    server_a.debug_luanch_test()

2023-09-19 11:08:40,142 |INFO     |beamngpy.BeamNGpy             |Loaded map.
2023-09-19 11:08:42,138 |INFO     |beamngpy.Vehicle              |Vehicle vehicleA connected to simulation.
2023-09-19 11:08:42,138 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA
2023-09-19 11:08:43,238 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-09-19 11:08:43,238 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-09-19 11:08:43,242 |INFO     |beamngpy.Scenario             |Connected to scenario: tag
2023-09-19 11:08:43,266 |INFO     |beamngpy.BeamNGpy             |Starting scenario.


In [7]:
CFG_FILE = ".\\configs\\pointpillar.yaml"

cfg = cfg_from_yaml_file(CFG_FILE, cfg)
cfg.DATA_CONFIG.DATA_PROCESSOR

[{'NAME': 'transform_points_to_voxels',
  'VOXEL_SIZE': [0.16, 0.16, 4],
  'MAX_POINTS_PER_VOXEL': 32,
  'MAX_NUMBER_OF_VOXELS': {'train': 16000, 'test': 40000}}]

In [8]:
client_b.debug_luanch_test(None, lidar_para={
    "requested_update_time": 0.01, 
    "is_visualised": True,
    "is_using_shared_memory": True,
    "vertical_resolution": 64,
    # "horizontal_angle": 90,
    # "dir":(-0.3, -0.5, 0), Not work.
})
pcs_dataset = beamng_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, lidar=client_b.lidar_t, class_names=cfg.CLASS_NAMES)
# model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=pcs_dataset)
# model.load_params_from_file(filename=CKPT_FILE, logger=logger, to_cpu=True)
object_tracker = mono_label_distance_tracker(track_length=10)
model = torch.jit.load("./torch_scripts/point_pillar_model.pt")
model.cuda()
model.eval()

2023-09-19 11:08:43,617 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA


tag
{'vehicleA': <beamngpy.vehicle.vehicle.Vehicle object at 0x000001FF1D4071F0>}


2023-09-19 11:08:45,340 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-09-19 11:08:45,341 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-09-19 11:08:45,406 |INFO     |beamngpy.Lidar                |Opened lidar: "lidar"


RecursiveScriptModule(
  original_name=PointPillar
  (vfe): RecursiveScriptModule(
    original_name=PillarVFE
    (pfn_layers): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=PFNLayer
        (linear): RecursiveScriptModule(original_name=Linear)
        (norm): RecursiveScriptModule(original_name=BatchNorm1d)
      )
    )
  )
  (map_to_bev_module): RecursiveScriptModule(original_name=PointPillarScatter)
  (backbone_2d): RecursiveScriptModule(
    original_name=BaseBEVBackbone
    (blocks): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=ZeroPad2d)
        (1): RecursiveScriptModule(original_name=Conv2d)
        (2): RecursiveScriptModule(original_name=BatchNorm2d)
        (3): RecursiveScriptModule(original_name=ReLU)
        (4): RecursiveScriptModule(original_name=Conv2d)
        (5): RecursiveScript

In [9]:
vis = o3d.visualization.Visualizer()
vis.create_window()

with torch.no_grad():
    for idx, data_dict in enumerate(pcs_dataset):
        pre_time = data_dict['pre_time']
        before_time = time.perf_counter()
        
        data_dict = pcs_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)

        pred_dicts, _ = model.forward(data_dict)
        
        cls_preds = pred_dicts["pred_scores"]
        box_preds = pred_dicts["pred_boxes"]
        label_preds = pred_dicts["pred_labels"]

        selected, selected_scores = class_agnostic_nms(
                            box_scores=cls_preds, box_preds=box_preds,
                            score_thresh=0.5
                        )

        final_scores = selected_scores
        final_labels = label_preds[selected]
        final_boxes = box_preds[selected]
        
            
        if final_boxes is not None:
            object_tracker.updates_object(final_boxes[np.newaxis, :, :])

        box = object_tracker.get_all_bounding_box()
        tracks = object_tracker.get_all_tracks()

        draw_scenes(vis,
            points=data_dict['points'][:, 1:], ref_boxes=box,
            ref_scores=None, ref_labels=None, confidence=None, tracks=tracks
        )
        
        
        
        after_time = time.perf_counter()
        
        
        clear_output(wait=True)
        logger.info(f"Compute time: {after_time - before_time:.3f} + {pre_time:.3f} == {after_time - before_time + pre_time:.3f}s")
        logger.info(f"Target amount: {len(final_boxes)}")
        logger.info(f"Current uuid: {object_tracker.get_last_uuid()}")
        
        # tensorized_final_boxes = torch.stack(final_boxes)[:, 0:3].view(1, -1, 3)
        # tesnorized_final_labels = torch.stack(final_labels).view(1, -1)

        # draw_scenes(vis,
        #         points=data_dict['points'][:, 1:], ref_boxes=pred_dicts['pred_boxes'].detach(),
        #         ref_scores=pred_dicts['pred_scores'].detach(), ref_labels=pred_dicts['pred_labels'].detach(), confidence=0.5
        #     )
        
        # draw_scenes(vis,
        #         points=data_dict['points'][:, 1:], ref_boxes=None,
        #         ref_scores=None, ref_labels=None, confidence=0.5
        #     )
        

        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries()


2023-09-19 11:09:17,148   INFO  Compute time: 0.177 + 0.008 == 0.186s
2023-09-19 11:09:17,149   INFO  Target amount: 3
2023-09-19 11:09:17,150   INFO  Current uuid: 26


## Open3d window is black screen-Open3d 黑屏

It‘s normal that the Open3d window initializes with black screen for the view is not initialized properly.

Open3d 刚打开是黑屏是正常现象，因为视角没有正常初始化。

Use `ctrl+C` to copy the following json and `ctrl+v` to paste it in the Open3d window.

用 `ctrl+C` 复制下面的 json 并使用`ctrl+v`粘贴到所打开的Open3d窗口中：

```json
{ 
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false, 
	"trajectory" :  
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}
```



(161070, 4)
(79241, 4)

In [ ]:
client_b.disconnect_client()


In [ ]:
server_a.close_server()

2023-08-23 15:20:43,846 |INFO     |beamngpy.BeamNGpy             |Closing BeamNGpy instance.
2023-08-23 15:20:43,846 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle vehicleA and detached sensors.
2023-08-23 15:20:43,847 |INFO     |beamngpy.BeamNGpy             |Terminating BeamNG.tech process.
2023-08-23 15:20:44,115 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_beampy_torch\lib\subprocess.py:1067: ResourceWarning: subprocess 24252 is still running
  _warn("subprocess %s is still running" % self.pid,

2023-08-23 15:20:44,115 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_beampy_torch\lib\subprocess.py:1067: ResourceWarning: subprocess 24252 is still running
  _warn("subprocess %s is still running" % self.pid,

2023-08-23 15:20:44,118 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_beampy_torch\lib\site-packages\beamngpy\beamng\beamng.py:283: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  self.process =